In [1]:
from IPython.display import Image, Markdown, display

import base64

In [2]:
import pandas as pd
import re

import os, sys
path_to_import = "../src/models/DMCAR"
if path_to_import not in sys.path:
    sys.path.append(path_to_import)
import tomermaid

print(sys.path)

['/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/tomk/environment/devpy/lib/python3.10/site-packages', '/home/tomk/Documents/Coding/gitHub/digraph/src', '../src/models/DMCAR']


In [3]:
def mm_ink(graphbytes):
  """Given a bytes object holding a Mermaid-format graph, return a URL that will generate the image."""
  base64_bytes = base64.b64encode(graphbytes)
  base64_string = base64_bytes.decode("ascii")
  return "https://mermaid.ink/img/" + base64_string

def mm_display(graphbytes):
  """Given a bytes object holding a Mermaid-format graph, display it."""
  display(Image(url=mm_ink(graphbytes)))

def mm(graph):
  """Given a string containing a Mermaid-format graph, display it."""
  graphbytes = graph.encode("ascii")
  mm_display(graphbytes)

def mm_link(graph):
  """Given a string containing a Mermaid-format graph, return URL for display."""
  graphbytes = graph.encode("ascii")
  return mm_ink(graphbytes)
  
def mm_path(path):
  """Given a path to a file containing a Mermaid-format graph, display it"""
  with open(path, 'rb') as f:
    graphbytes = f.read()
  mm_display(graphbytes)



In [4]:
e = "Payment"
attribs = [{"type" : "str", 
            "name" : "id", 
            "isPK" : "Yes", 
            "isFK" : "No", 
            "nulls" : "No", 
            "description" : "Unique Identifier"}, 
           {"type" : "str", 
            "name" : "type", 
            "isPK" : "No", 
            "isFK" : "No", 
            "nulls" : "No", 
            "description" : "Describes the type of the payment"}, 
           {"type" : "float", 
            "name" : "Amount", 
            "isPK" : "No", 
            "isFK" : "No", 
            "nulls" : "No", 
            "description" : "The amount of the payment in associated currency"}, 
           {"type" : "str", 
            "name" : "Currency", 
            "isPK" : "No", 
            "isFK" : "No", 
            "nulls" : "No", 
            "description" : "The 3-char currency code of the amount in the payment"}, 
          ]

In [5]:
print(tomermaid.format_erd_node(e, attribs) + "\n" + tomermaid.format_erd_relationship("JOIN", "LEFT", "RIGHT", "one", "no", "many", "yes"))

	Payment {
    		str id PK "Unique Identifier" 
		str type  "Describes the type of the payment" 
		float Amount  "The amount of the payment in associated currency" 
		str Currency  "The 3-char currency code of the amount in the payment" 
    	}
LEFT ||--o{ RIGHT : JOIN


In [6]:
mm("""erDiagram\n """ + tomermaid.format_erd_node(e, attribs) + "\n" + tomermaid.format_erd_relationship("JOIN", "LEFT", "RIGHT", "one", "no", "many", "yes"))

In [7]:
mm("""erDiagram 
	CAR ||--o{ NAMED-DRIVER : allows 
	CAR { 
		string registrationNumber PK 
		string make string model 
		string[] parts 
		} 
	PERSON ||--o{ NAMED-DRIVER : is 
	PERSON { 
		string driversLicense PK "The license #" 
		string(99) firstName "Only 99 characters are allowed" 
		string lastName 
		string phone UK int age 
		} 
	NAMED-DRIVER { 
		string carRegistrationNumber PK, FK 
		string driverLicence PK, FK 
		} 
	MANUFACTURER only one to zero or more CAR : makes""")

In [8]:
model_df = pd.read_excel("payment_example.xlsx")

In [9]:
import marshaller

In [10]:
classes=[]
for g in model_df.groupby("Class").groups:
    classes.append( marshaller.Class.marshal_from_DMCAR(model_df.groupby("Class").get_group(g).iloc[0]))

attributes=[]
for a in model_df.groupby(["Class", "Attribute"]).groups:
    if all([not pd.isna(v) for v in a]):
        attribute = marshaller.Attribute.marshal_from_DMCAR(model_df.groupby(["Class", "Attribute"]).get_group(a).iloc[0])
        attribute.set_parents(classes)
        attributes.append( attribute )

relationships=[]
for r in model_df.groupby(["Relationship"]).groups:
    print(r)
    relationship = marshaller.Relationship.marshal_from_DMCAR(model_df.groupby(["Relationship"]).get_group(r).iloc[0])
    relationship.set_parents(classes, attributes)
    relationships.append( relationship )


hostedByChannel
receivedOver
referencedInMedia


/tmp/ipykernel_524426/807506178.py:15: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  relationship = marshaller.Relationship.marshal_from_DMCAR(model_df.groupby(["Relationship"]).get_group(r).iloc[0])
/tmp/ipykernel_524426/807506178.py:15: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  relationship = marshaller.Relationship.marshal_from_DMCAR(model_df.groupby(["Relationship"]).get_group(r).iloc[0])
/tmp/ipykernel_524426/807506178.py:15: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  relationship = marshaller.Relationship.marshal_from_DMCA

In [11]:
relationships

[Relationship(namespace='IconPaymentsConceptual', name='hostedByChannel', label=nan, description='references the channel that handled the originating media', type=nan, from_namespace='IconPaymentsConceptual', from_class_name='Media', from_cardinality='One', from_cardinality_one=True, to_namespace='IconPaymentsConceptual', to_class_name='Channel', to_cardinality='Many', to_cardinality_one=False, from_class=Class(namespace='IconPaymentsConceptual', name='Media', label='Media', description='Media containing payment messaging', attributes=None), from_attribute_name=nan, from_attribute=None, to_class=Class(namespace='IconPaymentsConceptual', name='Channel', label='Channel', description='Text describing a channel', attributes=None), to_attribute_name=nan, to_attribute=None),
 Relationship(namespace='IconPaymentsConceptual', name='receivedOver', label=nan, description='describes the channel over which the payment was initially triggered', type='Conceptual', from_namespace='IconPaymentsConcept

In [12]:
relationships[0]

Relationship(namespace='IconPaymentsConceptual', name='hostedByChannel', label=nan, description='references the channel that handled the originating media', type=nan, from_namespace='IconPaymentsConceptual', from_class_name='Media', from_cardinality='One', from_cardinality_one=True, to_namespace='IconPaymentsConceptual', to_class_name='Channel', to_cardinality='Many', to_cardinality_one=False, from_class=Class(namespace='IconPaymentsConceptual', name='Media', label='Media', description='Media containing payment messaging', attributes=None), from_attribute_name=nan, from_attribute=None, to_class=Class(namespace='IconPaymentsConceptual', name='Channel', label='Channel', description='Text describing a channel', attributes=None), to_attribute_name=nan, to_attribute=None)

In [13]:
model_df[['Class','Attribute']]

,Class,Attribute
0,Payment,PaymentId
1,Channel,NaN
2,Media,NaN
3,Block,NaN
4,Entry,NaN
5,PartyRole,NaN
6,NaN,NaN


In [14]:
for a in attributes:
    a.set_parents(classes)

In [15]:
classes

[Class(namespace='IconPaymentsConceptual', name='Block', label='Block', description='Location within media where transaction is sourced/referenced', attributes=None),
 Class(namespace='IconPaymentsConceptual', name='Channel', label='Channel', description='Text describing a channel', attributes=None),
 Class(namespace='IconPaymentsConceptual', name='Entry', label='Entry', description='A booking entry made in an internal account', attributes=None),
 Class(namespace='IconPaymentsConceptual', name='Media', label='Media', description='Media containing payment messaging', attributes=None),
 Class(namespace='IconPaymentsConceptual', name='PartyRole', label='PartyRole', description='A confluence of a Party and a Role that party is playing in the context of this payment', attributes=None),
 Class(namespace='IconPaymentsConceptual', name='Payment', label='Payment', description='Something to do with a payment', attributes=None)]

In [16]:
attributes

[Attribute(parent_class_name='Payment', namespace='IconPaymentsConceptual', name='PaymentId', label='PaymentId', description='An identifier to uniquely reference an individual payment', sequence=0.0, datatype='String', nulls='No', ispk='Yes', parent_class=Class(namespace='IconPaymentsConceptual', name='Payment', label='Payment', description='Something to do with a payment', attributes=None))]

In [17]:
c,a,r = marshaller.from_pandas(model_df)

[]
[]
[]
[]
[]
[Attribute(parent_class_name='Payment', namespace='IconPaymentsConceptual', name='PaymentId', label='PaymentId', description='An identifier to uniquely reference an individual payment', sequence=0.0, datatype='String', nulls='No', ispk='Yes', parent_class=Class(namespace='IconPaymentsConceptual', name='Payment', label='Payment', description='Something to do with a payment', attributes=None))]


/home/tomk/Documents/Coding/gitHub/datamodels/IDE/../src/models/DMCAR/marshaller.py:256: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  relationship = Relationship.marshal_from_DMCAR(df.groupby(["Relationship"]).get_group(r).iloc[0])
/home/tomk/Documents/Coding/gitHub/datamodels/IDE/../src/models/DMCAR/marshaller.py:256: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  relationship = Relationship.marshal_from_DMCAR(df.groupby(["Relationship"]).get_group(r).iloc[0])
/home/tomk/Documents/Coding/gitHub/datamodels/IDE/../src/models/DMCAR/marshaller.py:256: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. 

In [18]:
c

[Class(namespace='IconPaymentsConceptual', name='Block', label='Block', description='Location within media where transaction is sourced/referenced', attributes=[]),
 Class(namespace='IconPaymentsConceptual', name='Channel', label='Channel', description='Text describing a channel', attributes=[]),
 Class(namespace='IconPaymentsConceptual', name='Entry', label='Entry', description='A booking entry made in an internal account', attributes=[]),
 Class(namespace='IconPaymentsConceptual', name='Media', label='Media', description='Media containing payment messaging', attributes=[]),
 Class(namespace='IconPaymentsConceptual', name='PartyRole', label='PartyRole', description='A confluence of a Party and a Role that party is playing in the context of this payment', attributes=[]),
 Class(namespace='IconPaymentsConceptual', name='Payment', label='Payment', description='Something to do with a payment', attributes=[Attribute(parent_class_name='Payment', namespace='IconPaymentsConceptual', name='Pay

In [19]:
a

[Attribute(parent_class_name='Payment', namespace='IconPaymentsConceptual', name='PaymentId', label='PaymentId', description='An identifier to uniquely reference an individual payment', sequence=0.0, datatype='String', nulls='No', ispk='Yes', parent_class=Class(namespace='IconPaymentsConceptual', name='Payment', label='Payment', description='Something to do with a payment', attributes=[...]))]

In [20]:
[(e, l.from_attribute, l.to_attribute) for e,l in enumerate(r)]

[(0, None, None), (1, None, None), (2, None, None)]

In [21]:
False * 1

0

In [22]:
True * 1

1

In [23]:
pk = False
fk = False
uk = True
option_keys = ["PK", "FK", "UK"]
",".join([k for e,k in enumerate(option_keys) if [pk,fk,uk][e]])


'UK'

In [24]:
nodes=[]
for n in c:
    nodes.append(marshaller.MermaidWrapper.format_erd_node(n))

rels=[]
for l in r:
    rels.append(marshaller.MermaidWrapper.format_erd_relationship(l))

In [25]:
l.from_attribute

In [26]:
classes[-1].set_child_attributes(attributes)

In [27]:
attributes

[Attribute(parent_class_name='Payment', namespace='IconPaymentsConceptual', name='PaymentId', label='PaymentId', description='An identifier to uniquely reference an individual payment', sequence=0.0, datatype='String', nulls='No', ispk='Yes', parent_class=Class(namespace='IconPaymentsConceptual', name='Payment', label='Payment', description='Something to do with a payment', attributes=[...]))]

In [28]:
a_list = [a for a in attributes if a.parent_class.name == 'Payment']

In [29]:
a_list

[Attribute(parent_class_name='Payment', namespace='IconPaymentsConceptual', name='PaymentId', label='PaymentId', description='An identifier to uniquely reference an individual payment', sequence=0.0, datatype='String', nulls='No', ispk='Yes', parent_class=Class(namespace='IconPaymentsConceptual', name='Payment', label='Payment', description='Something to do with a payment', attributes=[...]))]

In [30]:
classes[-1].attributes[0]

Attribute(parent_class_name='Payment', namespace='IconPaymentsConceptual', name='PaymentId', label='PaymentId', description='An identifier to uniquely reference an individual payment', sequence=0.0, datatype='String', nulls='No', ispk='Yes', parent_class=Class(namespace='IconPaymentsConceptual', name='Payment', label='Payment', description='Something to do with a payment', attributes=[...]))

In [31]:
nodes

['\tBlock[Block] {\n        \n        \t}',
 '\tChannel[Channel] {\n        \n        \t}',
 '\tEntry[Entry] {\n        \n        \t}',
 '\tMedia[Media] {\n        \n        \t}',
 '\tPartyRole[PartyRole] {\n        \n        \t}',
 '\tPayment[Payment] {\n        \t\tString PaymentId PK "An identifier to uniquely reference an individual payment" \n        \t}']

In [32]:
rels

['Media }o--|| Channel : hostedByChannel',
 'Payment |o--|{ Channel : receivedOver',
 'Payment |o--|{ Media : referencedInMedia']

In [35]:
print("""erDiagram\n """ + "\n".join(nodes) + "\n".join(rels))

erDiagram
 	Block[Block] {
        
        	}
	Channel[Channel] {
        
        	}
	Entry[Entry] {
        
        	}
	Media[Media] {
        
        	}
	PartyRole[PartyRole] {
        
        	}
	Payment[Payment] {
        		String PaymentId PK "An identifier to uniquely reference an individual payment" 
        	}Media }o--|| Channel : hostedByChannel
Payment |o--|{ Channel : receivedOver
Payment |o--|{ Media : referencedInMedia


In [36]:
mm("""erDiagram\n """ + "\n".join(nodes) + "\n".join(rels))